In [1]:
## Montar google drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# Cargar el archivo desde Google Drive
import pandas as pd
# Cargar datos de entrenamiento y prueba
df_train = pd.read_excel("/content/drive/MyDrive/MachineLearning/TRAIN/GPT/grouped_trainytrial.xlsx")
df_test = pd.read_excel("/content/drive/MyDrive/MachineLearning/TEST/grouped_data_test.xlsx")



In [7]:
import pandas as pd
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    balanced_accuracy_score, classification_report
)
import joblib

# === PREPROCESAMIENTO ===
label_encoder = LabelEncoder()
scaler = MinMaxScaler()

# Selección de características
columnas_seleccionadas = [
    'POS', 'NEU', 'NEG', 'tristeza', 'miedo', 'disgusto', 'enojo', 'sorpresa', 'alegria',
     'toxicity',
    'me', 'mi', 'yo',
    'num_palabras_mayusculas',
    'num_palabras_largas',
    'negaciones',
    'falta de motivacion',
   'aislamiento social',
    'pensamientos suicidas',

]

X_train = df_train[columnas_seleccionadas].copy()
X_train[['num_palabras_mayusculas', 'num_palabras_largas', 'negaciones']] = scaler.fit_transform(
    X_train[['num_palabras_mayusculas', 'num_palabras_largas', 'negaciones']]
)

y_train = label_encoder.fit_transform(df_train['label'])


# === SPLIT VALIDACIÓN ===
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, test_size=0.3, random_state=42
)


modelo_manual = LGBMClassifier(
    objective='multiclass',
    num_class=3,
    boosting_type='gbdt',
    n_estimators=100,
    num_leaves=31,
    learning_rate=0.05,
    feature_fraction=0.7,
    verbosity=-1,
    random_state=42
)
# === ENTRENAMIENTO ===
modelo_manual.fit(X_train_split, y_train_split)

# === PREDICCIÓN Y EVALUACIÓN ===
y_val_pred = modelo_manual.predict(X_val_split)

accuracy = accuracy_score(y_val_split, y_val_pred)
precision = precision_score(y_val_split, y_val_pred, average='weighted')
recall = recall_score(y_val_split, y_val_pred, average='weighted')
f1 = f1_score(y_val_split, y_val_pred, average='weighted')
balanced_accuracy = balanced_accuracy_score(y_val_split, y_val_pred)

# === RESULTADOS ===
print("\nResultados del modelo con hiperparámetros manuales:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Balanced Accuracy: {balanced_accuracy:.4f}")
print("\nReporte de Clasificación:")
print(classification_report(y_val_split, y_val_pred,target_names=["control", "depresion", "ansiedad"]))



# === GUARDAR MODELO Y TRANSFORMADORES ===
joblib.dump(modelo_manual, 'lgbm_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')
joblib.dump(columnas_seleccionadas, 'columnas_seleccionadas.pkl')
joblib.dump(label_encoder.classes_, 'clases_modelo.pkl')

print("\nModelo con hiperparámetros definidos manualmente guardado exitosamente.")



Resultados del modelo con hiperparámetros manuales:
Accuracy: 0.8219
Precision: 0.8226
Recall: 0.8219
F1 Score: 0.8222
Balanced Accuracy: 0.8097

Reporte de Clasificación:
              precision    recall  f1-score   support

     control       0.87      0.86      0.86        76
   depresion       0.77      0.78      0.78        51
    ansiedad       0.79      0.79      0.79        19

    accuracy                           0.82       146
   macro avg       0.81      0.81      0.81       146
weighted avg       0.82      0.82      0.82       146


Modelo con hiperparámetros definidos manualmente guardado exitosamente.


In [8]:
import pandas as pd
import joblib
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, classification_report
)

# Cargar el modelo y preprocesadores guardados
modelo_optimizado = joblib.load('lgbm_model.pkl')
scaler = joblib.load('scaler.pkl')
label_encoder = joblib.load('label_encoder.pkl')
columnas_seleccionadas = joblib.load('columnas_seleccionadas.pkl')

# Preprocesamiento del conjunto de prueba
X_test = df_test[columnas_seleccionadas].copy()

# Escalar las columnas numéricas
X_test.loc[:, ['num_palabras_mayusculas', 'num_palabras_largas', 'negaciones']] = scaler.transform(
    X_test[['num_palabras_mayusculas', 'num_palabras_largas', 'negaciones']]
)

# Codificar las etiquetas del conjunto de prueba
y_test = label_encoder.transform(df_test['label'])

# Hacer predicciones en el conjunto de prueba
y_pred = modelo_optimizado.predict(X_test)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)

# Imprimir métricas
print("\nResultados en el conjunto de prueba:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Balanced Accuracy: {balanced_accuracy:.4f}")

# Reporte de clasificación detallado
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))



Resultados en el conjunto de prueba:
Accuracy: 0.8175
Precision: 0.8200
Recall: 0.8175
F1 Score: 0.8148
Balanced Accuracy: 0.7767

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.88      0.94      0.91       200
           1       0.68      0.74      0.71       100
           2       0.84      0.65      0.73       100

    accuracy                           0.82       400
   macro avg       0.80      0.78      0.78       400
weighted avg       0.82      0.82      0.81       400



<ipython-input-8-0e1d69c19bfa>:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.01128668 0.00225734 0.02257336 0.         0.00451467 0.04063205
 0.00451467 0.         0.01580135 0.00677201 0.03611738 0.00902935
 0.00677201 0.00902935 0.00677201 0.00902935 0.00451467 0.00677201
 0.00225734 0.00451467 0.01580135 0.02708804 0.00902935 0.00225734
 0.00902935 0.01354402 0.02934537 0.00677201 0.00902935 0.00225734
 0.03837472 0.00225734 0.00451467 0.0248307  0.02031603 0.02031603
 0.01128668 0.01805869 0.02031603 0.00451467 0.01354402 0.00225734
 0.00677201 0.00225734 0.01128668 0.00451467 0.00225734 0.00451467
 0.00451467 0.01128668 0.01128668 0.00451467 0.00225734 0.01128668
 0.0248307  0.         0.04288939 0.01805869 0.01128668 0.01128668
 0.02708804 0.01805869 0.01128668 0.0248307  0.00225734 0.
 0.01128668 0.00225734 0.02708804 0.01128668 0.02708804 0.00677201
 0.         0.         0.00451467 0.         0.0338